<a href="https://colab.research.google.com/github/opalinav/Transformers-for-Drug-Generation/blob/main/ChemBERTa_Antibiotic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ChemBERTA: Antibiotic Dataset



# Setup


In [ ]:
!curl -Lo conda_installer.py https://raw.githubusercontent.com/deepchem/deepchem/master/scripts/colab_install.py
import conda_installer
conda_installer.install()
!/root/miniconda/bin/conda info -e


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3457  100  3457    0     0  16958      0 --:--:-- --:--:-- --:--:-- 16946


add /root/miniconda/lib/python3.10/site-packages to PYTHONPATH
INFO:conda_installer:add /root/miniconda/lib/python3.10/site-packages to PYTHONPATH
python version: 3.10.12
INFO:conda_installer:python version: 3.10.12
fetching installer from https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
INFO:conda_installer:fetching installer from https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
done
INFO:conda_installer:done
installing miniconda to /root/miniconda
INFO:conda_installer:installing miniconda to /root/miniconda
done
INFO:conda_installer:done
installing openmm, pdbfixer
INFO:conda_installer:installing openmm, pdbfixer
added conda-forge to channels
INFO:conda_installer:added conda-forge to channels
done
INFO:conda_installer:done
conda packages installation finished!
INFO:conda_installer:conda packages installation finished!


# conda environments:
#
base                     /root/miniconda



In [ ]:
!pip install --pre deepchem
import deepchem
deepchem.__version__

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 940.8/940.8 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 17.3 MB/s eta 0:00:00


Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


'2.7.2.dev'

In [ ]:
!git clone https://github.com/NVIDIA/apex
!cd /content/apex
!pip install -v --no-cache-dir /content/apex
!pip install transformers
!pip install simpletransformers
!pip install wandb
!cd ..

Cloning into 'apex'...
remote: Enumerating objects: 11537, done.
remote: Counting objects: 100% (3630/3630), done.
remote: Compressing objects: 100% (520/520), done.
remote: Total 11537 (delta 3305), reused 3210 (delta 3107), pack-reused 7907
Receiving objects: 100% (11537/11537), 15.43 MiB | 10.75 MiB/s, done.
Resolving deltas: 100% (8107/8107), done.
Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
Processing ./apex
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.5/819.5 kB 4.0 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 6.9 MB/s eta 0:00:00
    Creating /tmp/pip-build-env-0z7iv6ef/overlay/local/bin
    changing mode of /tmp/pip-build-env-0z7iv6ef/overlay/local/bin/wheel to 755
  ERROR: pip's dependency resolver does not currently take into account all the packages that are

In [ ]:
import sys
!test -d bertviz_repo && echo "FYI: bertviz_repo directory already exists, to pull latest version uncomment this line: !rm -r bertviz_repo"
# !rm -r bertviz_repo # Uncomment if you need a clean pull from repo
!test -d bertviz_repo || git clone https://github.com/jessevig/bertviz bertviz_repo
if not 'bertviz_repo' in sys.path:
  sys.path += ['bertviz_repo']
!pip install regex

Cloning into 'bertviz_repo'...
remote: Enumerating objects: 1633, done.
remote: Counting objects: 100% (329/329), done.
remote: Compressing objects: 100% (118/118), done.
remote: Total 1633 (delta 231), reused 225 (delta 211), pack-reused 1304
Receiving objects: 100% (1633/1633), 198.37 MiB | 13.82 MiB/s, done.
Resolving deltas: 100% (1073/1073), done.


Cloning repo for ChemBERTa and MolNet dataloader

In [ ]:
!git clone https://github.com/seyonechithrananda/bert-loves-chemistry.git

Cloning into 'bert-loves-chemistry'...
remote: Enumerating objects: 1566, done.
remote: Counting objects: 100% (202/202), done.
remote: Compressing objects: 100% (111/111), done.
remote: Total 1566 (delta 96), reused 91 (delta 91), pack-reused 1364
Receiving objects: 100% (1566/1566), 55.35 MiB | 10.20 MiB/s, done.
Resolving deltas: 100% (1000/1000), done.
Updating files: 100% (122/122), done.


In [ ]:
!nvidia-smi

Sat Dec 16 03:54:20 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Antibiotic dataset

In [ ]:
import os

import numpy as np
import pandas as pd

import torch

In [ ]:
df = pd.read_csv('antibiotic_smile.csv')
# df = pd.read_csv('the_main_dataset.csv')
# make the first row the headers
# df.columns = df.iloc[0]
# df = df[1:]

In [ ]:
df

,Mean_Inhibition,SMILES,Name,Activity,NaN,NaN
1,0.041572,Cc1cc(O)c(C(=O)NC(C(=O)NC2C(=O)N3C(C(=O)O)=C(C...,CEFPIRAMIDE,Active,NaN,NaN
2,0.041876,CON=C1CN(c2nc3c(cc2F)c(=O)c(C(=O)O)cn3C2CC2)CC...,GEMIFLOXACIN MESYLATE,Active,NaN,NaN
3,0.041916,CCC(C)CCCCC(=O)NC(CCN)C(=O)NC(C(=O)NC(CCN)C(=O...,POLYMYXIN B SULFATE,Active,NaN,NaN
4,0.041964,Cl.N=C(N)n1cccn1,PRAXADINE HYDROCHLORIDE,Active,NaN,NaN
5,0.0422955,Cl.Cl.N=C(NCCCCCCNC(=N)NC(=N)Nc1ccc(Cl)cc1)NC(...,CHLORHEXIDINE DIHYDROCHLORIDE,Active,NaN,NaN
...,...,...,...,...,...,...
2331,1.23535,C=CC1(C)CC(=O)C2(O)C(C)(O1)C(OC(C)=O)C(O)C1C(C...,COLFORSIN,Inactive,NaN,NaN
2332,1.25165,O=S(=O)([O-])c1cc(O)c2c(N=Nc3ccc(Nc4ccccc4)c4c...,ANAZOLENE SODIUM,Inactive,NaN,NaN
2333,1.27315,Nc1ccc(S(=O)(=O)c2ccc(N)cc2)cc1,DAPSONE,Inactive,NaN,NaN
2334,2.2632,Cc1cc(-c2ccc(N=Nc3ccc4c(S(=O)(=O)[O-])cc(S(=O)...,EVANS BLUE,Inactive,NaN,NaN


In [ ]:
df["Activity"] = df["Activity"].map({"Active": 1, "Inactive": 0})

In [ ]:
df_classification = df[["SMILES", "Activity"]]

In [ ]:
df_classification

,SMILES,Activity
1,Cc1cc(O)c(C(=O)NC(C(=O)NC2C(=O)N3C(C(=O)O)=C(C...,1
2,CON=C1CN(c2nc3c(cc2F)c(=O)c(C(=O)O)cn3C2CC2)CC...,1
3,CCC(C)CCCCC(=O)NC(CCN)C(=O)NC(C(=O)NC(CCN)C(=O...,1
4,Cl.N=C(N)n1cccn1,1
5,Cl.Cl.N=C(NCCCCCCNC(=N)NC(=N)Nc1ccc(Cl)cc1)NC(...,1
...,...,...
2331,C=CC1(C)CC(=O)C2(O)C(C)(O1)C(OC(C)=O)C(O)C1C(C...,0
2332,O=S(=O)([O-])c1cc(O)c2c(N=Nc3ccc(Nc4ccccc4)c4c...,0
2333,Nc1ccc(S(=O)(=O)c2ccc(N)cc2)cc1,0
2334,Cc1cc(-c2ccc(N=Nc3ccc4c(S(=O)(=O)[O-])cc(S(=O)...,0


In [ ]:
df_classification = df_classification.rename(columns={"SMILES": "text", "Activity": "labels"})

In [ ]:
df_classification.dropna(inplace=True)
df_classification


,text,labels
1,Cc1cc(O)c(C(=O)NC(C(=O)NC2C(=O)N3C(C(=O)O)=C(C...,1
2,CON=C1CN(c2nc3c(cc2F)c(=O)c(C(=O)O)cn3C2CC2)CC...,1
3,CCC(C)CCCCC(=O)NC(CCN)C(=O)NC(C(=O)NC(CCN)C(=O...,1
4,Cl.N=C(N)n1cccn1,1
5,Cl.Cl.N=C(NCCCCCCNC(=N)NC(=N)Nc1ccc(Cl)cc1)NC(...,1
...,...,...
2331,C=CC1(C)CC(=O)C2(O)C(C)(O1)C(OC(C)=O)C(O)C1C(C...,0
2332,O=S(=O)([O-])c1cc(O)c2c(N=Nc3ccc(Nc4ccccc4)c4c...,0
2333,Nc1ccc(S(=O)(=O)c2ccc(N)cc2)cc1,0
2334,Cc1cc(-c2ccc(N=Nc3ccc4c(S(=O)(=O)[O-])cc(S(=O)...,0


In [ ]:
from sklearn.model_selection import train_test_split

def split_stratified_into_train_val_test(df_input, stratify_colname='y',
                                         frac_train=0.6, frac_val=0.15, frac_test=0.25,
                                         random_state=None):
    '''
    Splits a Pandas dataframe into three subsets (train, val, and test)
    following fractional ratios provided by the user, where each subset is
    stratified by the values in a specific column (that is, each subset has
    the same relative frequency of the values in the column). It performs this
    splitting by running train_test_split() twice.

    Parameters
    ----------
    df_input : Pandas dataframe
        Input dataframe to be split.
    stratify_colname : str
        The name of the column that will be used for stratification. Usually
        this column would be for the label.
    frac_train : float
    frac_val   : float
    frac_test  : float
        The ratios with which the dataframe will be split into train, val, and
        test data. The values should be expressed as float fractions and should
        sum to 1.0.
    random_state : int, None, or RandomStateInstance
        Value to be passed to train_test_split().

    Returns
    -------
    df_train, df_val, df_test :
        Dataframes containing the three splits.
    '''

    if frac_train + frac_val + frac_test != 1.0:
        raise ValueError('fractions %f, %f, %f do not add up to 1.0' % \
                         (frac_train, frac_val, frac_test))

    if stratify_colname not in df_input.columns:
        raise ValueError('%s is not a column in the dataframe' % (stratify_colname))

    X = df_input # Contains all columns.
    y = df_input[[stratify_colname]] # Dataframe of just the column on which to stratify.

    # Split original dataframe into train and temp dataframes.
    df_train, df_temp, y_train, y_temp = train_test_split(X,
                                                          y,
                                                          stratify=y,
                                                          test_size=(1.0 - frac_train),
                                                          random_state=random_state)

    # Split the temp dataframe into val and test dataframes.
    relative_frac_test = frac_test / (frac_val + frac_test)
    df_val, df_test, y_val, y_test = train_test_split(df_temp,
                                                      y_temp,
                                                      stratify=y_temp,
                                                      test_size=relative_frac_test,
                                                      random_state=random_state)

    assert len(df_input) == len(df_train) + len(df_val) + len(df_test)

    return df_train, df_val, df_test

In [ ]:
antibiotic_train_df, antibiotic_valid_df, antibiotic_test_df = split_stratified_into_train_val_test(df_classification, stratify_colname="labels", frac_train=0.8, frac_val=0.1, frac_test=0.1)

# Testing a pre-trained ChemBERTa on the antibiotic dataset

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs

model_pretrained = ClassificationModel('roberta', 'seyonec/PubChem10M_SMILES_BPE_396_250', args={'wandb_project': 'antibiotic-pretrained', 'evaluate_each_epoch': True, 'evaluate_during_training_verbose': True, 'no_save': True, 'num_train_epochs': 15, 'auto_weights': True})


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at seyonec/PubChem10M_SMILES_BPE_396_250 and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
!wandb login

wandb: Currently logged in as: clarise (nlpsquad). Use `wandb login --relogin` to force relogin


In [ ]:
import sklearn

# accuracy
result, model_outputs, wrong_predictions = model_pretrained.eval_model(antibiotic_test_df, acc=sklearn.metrics.accuracy_score)
print(result)

# ROC-PRC
result, model_outputs, wrong_predictions = model_pretrained.eval_model(antibiotic_test_df, acc=sklearn.metrics.average_precision_score)
print(result)

  0%|          | 0/234 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/30 [00:00<?, ?it/s]

{'mcc': -0.16750086484657414, 'tp': 3, 'tn': 84, 'fp': 138, 'fn': 9, 'auroc': 0.2865990990990991, 'auprc': 0.03674844218255045, 'acc': 0.3717948717948718, 'eval_loss': 0.73355712890625}


  0%|          | 0/234 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/30 [00:00<?, ?it/s]

{'mcc': -0.16750086484657414, 'tp': 3, 'tn': 84, 'fp': 138, 'fn': 9, 'auroc': 0.2865990990990991, 'auprc': 0.03674844218255045, 'acc': 0.04378068739770868, 'eval_loss': 0.73355712890625}


# Fine-tuning ChemBERTa on the antibiotic dataset

In [ ]:
from simpletransformers.classification import ClassificationModel
import logging
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

## BPE tokenizer

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs

model = ClassificationModel('roberta', 'seyonec/PubChem10M_SMILES_BPE_396_250', args={'evaluate_each_epoch': True, 'evaluate_during_training_verbose': True, 'no_save': True, 'num_train_epochs': 15, 'auto_weights': True}) # You can set class weights by using the optional weight argument


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at seyonec/PubChem10M_SMILES_BPE_396_250 and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
print(model.tokenizer)

RobertaTokenizerFast(name_or_path='seyonec/PubChem10M_SMILES_BPE_396_250', vocab_size=7924, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	4: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True, special=True),
}


In [ ]:
print(f"Train Dataset: {antibiotic_train_df.shape}")
print(f"Eval Dataset: {antibiotic_valid_df.shape}")
print(f"TEST Dataset: {antibiotic_test_df.shape}")

Train Dataset: (1868, 2)
Eval Dataset: (233, 2)
TEST Dataset: (234, 2)


In [ ]:
!wandb login

wandb: Currently logged in as: clarise (nlpsquad). Use `wandb login --relogin` to force relogin


In [ ]:
# Create directory to store model weights (change path accordingly to where you want!)
!mkdir BPE_PubChem_10M_Antibiotic_run

# Train the model , 'overwrite_output_dir': True
model.train_model(antibiotic_train_df, eval_df=antibiotic_valid_df, output_dir='/content/BPE_PubChem_10M_Antibiotic_run', args={'wandb_project': 'antibiotic-bpe'})


  0%|          | 0/1868 [00:00<?, ?it/s]

Epoch:   0%|          | 0/15 [00:00<?, ?it/s]

Running Epoch 0 of 15:   0%|          | 0/234 [00:00<?, ?it/s]

Running Epoch 1 of 15:   0%|          | 0/234 [00:00<?, ?it/s]

Running Epoch 2 of 15:   0%|          | 0/234 [00:00<?, ?it/s]

...

Running Epoch 11 of 15:   0%|          | 0/234 [00:00<?, ?it/s]

Running Epoch 12 of 15:   0%|          | 0/234 [00:00<?, ?it/s]

Running Epoch 13 of 15:   0%|          | 0/234 [00:00<?, ?it/s]

Running Epoch 14 of 15:   0%|          | 0/234 [00:00<?, ?it/s]

(3510, 0.042684849090555796)

In [ ]:
import sklearn

# accuracy
result, model_outputs, wrong_predictions = model.eval_model(antibiotic_test_df, acc=sklearn.metrics.accuracy_score)
print(result)

# ROC-PRC
result, model_outputs, wrong_predictions = model.eval_model(antibiotic_test_df, acc=sklearn.metrics.average_precision_score)
print(result)

  0%|          | 0/234 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/30 [00:00<?, ?it/s]

Training loss,▁▆▁▁▁█▁▁▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
lr,▃▄████▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
Training loss,2e-05
global_step,3500
lr,0.0


{'mcc': 0.7026160243580875, 'tp': 7, 'tn': 221, 'fp': 1, 'fn': 5, 'auroc': 0.9583333333333334, 'auprc': 0.7584719879588634, 'acc': 0.9743589743589743, 'eval_loss': 0.220645934343338}


  0%|          | 0/234 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/30 [00:00<?, ?it/s]

{'mcc': 0.7026160243580875, 'tp': 7, 'tn': 221, 'fp': 1, 'fn': 5, 'auroc': 0.9583333333333334, 'auprc': 0.7584719879588634, 'acc': 0.5317841880341881, 'eval_loss': 0.220645934343338}


In [ ]:
# res=pd.read_csv('results.csv')

In [ ]:
def predict_activity(smile):
  predictions,raw=model.predict([smile])
  if predictions[0]==0:
    return 'Inactive'
  else:
    return 'Active'


In [ ]:
# res['active'] = res['smiles'].apply(predict_activity)

In [ ]:
# res['smiles'].loc[res['active']!='Inactive']

In [ ]:
# res.to_csv('results_active.csv')

In [ ]:
# should predict activity level = 1
predictions, raw_outputs = model.predict(["O=c1c2c(oc3ccccc13)=C=CC(c1ccccc1)=CC=2"])

In [ ]:
print(predictions)
print(raw_outputs)

In [ ]:
# Cc1cc(O)c(C(=O)NC(C(=O)NC2C(=O)N3C(C(=O)O)=C(CSc4nnnn4C)CSC23)c2ccc(O)cc2)cn1
predictions, raw_outputs = model.predict(["Cc1cc(O)c(C(=O)NC(C(=O)NC2C(=O)N3C(C(=O)O)=C(CSc4nnnn4C)CSC23)c2ccc(O)cc2)cn1"])

In [ ]:
print(predictions)
print(raw_outputs)

In [ ]:
# CC1=CC(O)=C(C)NC(=C(C(=O)NC2C(=O)N3C(C(=O)O)NC(CSc4nnnn4C)CSC23)c2ccc(O)cc2)C=N1
predictions, raw_outputs = model.predict(["CC1=CC(O)=C(C)NC(=C(C(=O)NC2C(=O)N3C(C(=O)O)NC(CSc4nnnn4C)CSC23)c2ccc(O)cc2)C=N1"])

In [ ]:
print(predictions)
print(raw_outputs)

## SMILES tokenizer

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs

model = ClassificationModel('roberta', 'seyonec/SMILES_tokenized_PubChem_shard00_160k', args={'evaluate_each_epoch': True, 'evaluate_during_training_verbose': True, 'no_save': True, 'num_train_epochs': 15, 'auto_weights': True}) # You can set class weights by using the optional weight argument

config.json:   0%|          | 0.00/515 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/336M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at seyonec/SMILES_tokenized_PubChem_shard00_160k and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/327 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/8.14k [00:00<?, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
print(model.tokenizer)

RobertaTokenizerFast(name_or_path='seyonec/SMILES_tokenized_PubChem_shard00_160k', vocab_size=591, model_max_length=514, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	11: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	12: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	13: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	14: AddedToken("[MASK]", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
	591: AddedToken("<s>", rstrip=False, lstrip=False, single_word=Fals

In [ ]:
print(f"Train Dataset: {antibiotic_train_df.shape}")
print(f"Eval Dataset: {antibiotic_valid_df.shape}")
print(f"TEST Dataset: {antibiotic_test_df.shape}")

Train Dataset: (1868, 2)
Eval Dataset: (233, 2)
TEST Dataset: (234, 2)


In [ ]:
!wandb login

wandb: Currently logged in as: clarise (nlpsquad). Use `wandb login --relogin` to force relogin


In [ ]:
# Create directory to store model weights
!mkdir SmilesTokenizer_PubChem_10M_Antibiotic_run

# Train the model
model.train_model(antibiotic_train_df, eval_df=antibiotic_valid_df, output_dir='/content/SmilesTokenizer_PubChem_10M_Antibiotic_run', args={'wandb_project': 'antibiotic-smiles'})


  0%|          | 0/1868 [00:00<?, ?it/s]

Epoch:   0%|          | 0/15 [00:00<?, ?it/s]

Running Epoch 0 of 15:   0%|          | 0/234 [00:00<?, ?it/s]

Running Epoch 1 of 15:   0%|          | 0/234 [00:00<?, ?it/s]

Running Epoch 2 of 15:   0%|          | 0/234 [00:00<?, ?it/s]

...

Running Epoch 11 of 15:   0%|          | 0/234 [00:00<?, ?it/s]

Running Epoch 12 of 15:   0%|          | 0/234 [00:00<?, ?it/s]

Running Epoch 13 of 15:   0%|          | 0/234 [00:00<?, ?it/s]

Running Epoch 14 of 15:   0%|          | 0/234 [00:00<?, ?it/s]

(3510, 0.05089261106446258)

In [ ]:
import sklearn

# accuracy
result, model_outputs, wrong_predictions = model.eval_model(antibiotic_test_df, acc=sklearn.metrics.accuracy_score)
print(result)

# ROC-PRC
result, model_outputs, wrong_predictions = model.eval_model(antibiotic_test_df, acc=sklearn.metrics.average_precision_score)
print(result)

  0%|          | 0/234 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/30 [00:00<?, ?it/s]

Training loss,▁▆▁▁▁▃▁▇▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
lr,▃▄████▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
Training loss,1e-05
global_step,3500
lr,0.0


{'mcc': 0.5751646433835945, 'tp': 5, 'tn': 221, 'fp': 1, 'fn': 7, 'auroc': 0.8641141141141141, 'auprc': 0.6708014538957767, 'acc': 0.9658119658119658, 'eval_loss': 0.23598676472902297}


  0%|          | 0/234 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/30 [00:00<?, ?it/s]

{'mcc': 0.5751646433835945, 'tp': 5, 'tn': 221, 'fp': 1, 'fn': 7, 'auroc': 0.8641141141141141, 'auprc': 0.6708014538957767, 'acc': 0.3771367521367522, 'eval_loss': 0.23598676472902297}


In [ ]:
# Lets input a molecule with an activity value of 1
predictions, raw_outputs = model.predict(["O=C1C=C2C(=CCOC2O)O1"])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
print(predictions)
print(raw_outputs)

[1]
[[-5.23046875  5.16796875]]


In [ ]:
# Cc1cc(O)c(C(=O)NC(C(=O)NC2C(=O)N3C(C(=O)O)=C(CSc4nnnn4C)CSC23)c2ccc(O)cc2)cn1
predictions, raw_outputs = model.predict(["Cc1cc(O)c(C(=O)NC(C(=O)NC2C(=O)N3C(C(=O)O)=C(CSc4nnnn4C)CSC23)c2ccc(O)cc2)cn1"])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
print(predictions)
print(raw_outputs)

[1]
[[-5.3359375   5.26171875]]


In [ ]:
# CC1=CC(O)=C(C)NC(=C(C(=O)NC2C(=O)N3C(C(=O)O)NC(CSc4nnnn4C)CSC23)c2ccc(O)cc2)C=N1
predictions, raw_outputs = model.predict(["CC1=CC(O)=C(C)NC(=C(C(=O)NC2C(=O)N3C(C(=O)O)NC(CSc4nnnn4C)CSC23)c2ccc(O)cc2)C=N1"])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
print(predictions)
print(raw_outputs)

[1]
[[-1.92675781  1.49121094]]
